In [ ]:
import urllib.request
import urllib.error
import psutil
import psycopg2
import logging
from typing import List, Optional, Any, Union
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage

# Configure logging for better error tracking
logger = logging.getLogger(__name__)

# Global database connection variables
connection: Optional[psycopg2.extensions.connection] = None
cursor: Optional[psycopg2.extensions.cursor] = None

# Database configuration - consider moving to environment variables
DB_CONFIG = {
    "dbname": "mnemnosyne",
    "user": "postgres", 
    "password": "kitten samsara kaboodle nirvana",
    "host": "localhost",
    "port": "5432"
}

@tool
def scrape(url: str) -> str:
    """
    Scrape visible text content from a website URL.
    
    Args:
        url (str): The URL to scrape. Must be a valid HTTP/HTTPS URL.
        
    Returns:
        str: The visible text content wrapped in <text> tags, or error message.
        
    Example:
        >>> scrape("https://example.com")
        "<text>Example Domain This domain is for use in illustrative examples...</text>"
    
    Note:
        - Only scrapes visible text, not HTML markup
        - Has basic error handling for network issues
        - Returns content wrapped in XML-style tags for parsing
    """
    try:
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url
            
        with urllib.request.urlopen(url, timeout=30) as response:
            html = response.read()
            
        # Assuming text_from_html is defined elsewhere
        visible_text = text_from_html(html)
        return f"<text>{visible_text}</text>"
        
    except urllib.error.URLError as e:
        logger.error(f"URL error while scraping {url}: {e}")
        return f"URL error occurred: {e}"
    except urllib.error.HTTPError as e:
        logger.error(f"HTTP error while scraping {url}: {e}")
        return f"HTTP error occurred: {e.code} - {e.reason}"
    except Exception as e:
        logger.error(f"Unexpected error while scraping {url}: {e}")
        return f"An unexpected error occurred: {e}"

@tool
def get_cpu_usage() -> str:
    """
    Get current CPU utilization percentage.
    
    Returns:
        str: CPU usage as a percentage string (e.g., "25.4").
        
    Example:
        >>> get_cpu_usage()
        "15.2"
        
    Note:
        - Returns instantaneous CPU usage (no interval averaging)
        - Single aggregate value across all CPU cores
        - May return "0.0" on very fast calls due to no interval measurement
    """
    try:
        cpu_percent = psutil.cpu_percent(interval=None, percpu=False)
        return str(cpu_percent)
    except psutil.Error as e:
        logger.error(f"psutil error getting CPU usage: {e}")
        return f"System monitoring error occurred: {e}"
    except Exception as e:
        logger.error(f"Unexpected error getting CPU usage: {e}")
        return f"An unexpected error occurred: {e}"

@tool
def exec_query(query: str) -> str:
    """
    Execute a SQL query against the PostgreSQL database.
    
    Args:
        query (str): The SQL query to execute. Should be properly formatted SQL.
        
    Returns:
        str: Success message or error description.
        
    Example:
        >>> exec_query("INSERT INTO users (name) VALUES ('Alice')")
        "Query executed successfully"
        
    Warning:
        - Ensure database connection is established first
        - Use commit() to persist changes for INSERT/UPDATE/DELETE operations
        - Be cautious with destructive operations
    """
    global cursor
    
    if cursor is None:
        return "Error: No database connection established. Use connect() first."
        
    try:
        cursor.execute(query)
        return "Query executed successfully"
    except psycopg2.Error as e:
        logger.error(f"PostgreSQL error executing query: {e}")
        return f"Database error occurred: {e}"
    except Exception as e:
        logger.error(f"Unexpected error executing query: {e}")
        return f"An unexpected error occurred: {e}"

@tool
def exec_many(query: str, vars_list: List[Any]) -> str:
    """
    Execute a SQL query multiple times with different parameter sets.
    
    Args:
        query (str): The SQL query with parameter placeholders (e.g., %s, %(name)s).
        vars_list (List[Any]): List of parameter tuples/dicts for each execution.
        
    Returns:
        str: Success message or error description.
        
    Example:
        >>> exec_many("INSERT INTO users (name, age) VALUES (%s, %s)", 
        ...           [('Alice', 30), ('Bob', 25)])
        "Batch query executed successfully for 2 rows"
        
    Note:
        - More efficient than multiple individual execute() calls
        - Use commit() to persist changes
        - All executions are part of the same transaction
    """
    global cursor
    
    if cursor is None:
        return "Error: No database connection established. Use connect() first."
        
    if not isinstance(vars_list, list):
        return "Error: vars_list must be a list of parameter sets"
        
    try:
        cursor.executemany(query, vars_list)
        return f"Batch query executed successfully for {len(vars_list)} rows"
    except psycopg2.Error as e:
        logger.error(f"PostgreSQL error executing batch query: {e}")
        return f"Database error occurred: {e}"
    except Exception as e:
        logger.error(f"Unexpected error executing batch query: {e}")
        return f"An unexpected error occurred: {e}"

@tool
def fetch_all() -> Union[List[Any], str]:
    """
    Fetch all remaining rows from the last executed query.
    
    Returns:
        Union[List[Any], str]: List of result rows, or error message.
        
    Example:
        >>> exec_query("SELECT name, age FROM users")
        >>> fetch_all()
        [('Alice', 30), ('Bob', 25)]
        
    Note:
        - Must be called after a SELECT query
        - Returns empty list if no rows match
        - Consumes all remaining results from cursor
    """
    global cursor
    
    if cursor is None:
        return "Error: No database connection established. Use connect() first."
        
    try:
        results = cursor.fetchall()
        return results if results is not None else []
    except psycopg2.Error as e:
        logger.error(f"PostgreSQL error fetching results: {e}")
        return f"Database error occurred: {e}"
    except Exception as e:
        logger.error(f"Unexpected error fetching results: {e}")
        return f"An unexpected error occurred: {e}"

@tool
def fetch_one() -> Union[Any, str]:
    """
    Fetch the next row from the last executed query.
    
    Returns:
        Union[Any, str]: Single result row, None if no more rows, or error message.
        
    Example:
        >>> exec_query("SELECT name FROM users LIMIT 1")
        >>> fetch_one()
        ('Alice',)
        
    Note:
        - Must be called after a SELECT query
        - Returns None when no more rows available
        - Can be called multiple times to iterate through results
    """
    global cursor
    
    if cursor is None:
        return "Error: No database connection established. Use connect() first."
        
    try:
        result = cursor.fetchone()
        return result
    except psycopg2.Error as e:
        logger.error(f"PostgreSQL error fetching single result: {e}")
        return f"Database error occurred: {e}"
    except Exception as e:
        logger.error(f"Unexpected error fetching single result: {e}")
        return f"An unexpected error occurred: {e}"

@tool
def reconnect() -> str:
    """
    Close current database connection and establish a new one.
    
    Returns:
        str: Success message or error description.
        
    Example:
        >>> reconnect()
        "Database reconnected successfully"
        
    Note:
        - Useful for recovering from connection timeouts
        - Any uncommitted transactions will be lost
        - Automatically handles cleanup of old connection
    """
    try:
        close_result = close()
        if "error" in close_result.lower():
            logger.warning(f"Warning during close: {close_result}")
            
        connect_result = connect()
        if "error" in connect_result.lower():
            return connect_result
            
        return "Database reconnected successfully"
    except Exception as e:
        logger.error(f"Unexpected error during reconnection: {e}")
        return f"An unexpected error occurred during reconnection: {e}"

@tool
def close() -> str:
    """
    Close the current database connection and cursor.
    
    Returns:
        str: Success message or error description.
        
    Example:
        >>> close()
        "Database connection closed successfully"
        
    Warning:
        - Any uncommitted transactions will be lost
        - Call commit() before closing to save changes
        - Connection must be re-established with connect() for further operations
    """
    global cursor, connection
    
    try:
        if cursor:
            cursor.close()
            cursor = None
            
        if connection:
            connection.close()
            connection = None
            
        return "Database connection closed successfully"
    except psycopg2.Error as e:
        logger.error(f"PostgreSQL error closing connection: {e}")
        return f"Database error occurred while closing: {e}"
    except Exception as e:
        logger.error(f"Unexpected error closing connection: {e}")
        return f"An unexpected error occurred while closing: {e}"

@tool
def connect() -> str:
    """
    Establish a connection to the PostgreSQL database.
    
    Returns:
        str: Success message or error description.
        
    Example:
        >>> connect()
        "Database connected successfully"
        
    Note:
        - Connects to the 'mnemnosyne' database with predefined credentials
        - Automatically creates a cursor for query execution
        - Consider using environment variables for sensitive credentials
    """
    global cursor, connection
    
    try:
        connection = psycopg2.connect(**DB_CONFIG)
        cursor = connection.cursor()
        return "Database connected successfully"
    except psycopg2.OperationalError as e:
        logger.error(f"PostgreSQL operational error during connection: {e}")
        return f"Database connection failed: {e}"
    except psycopg2.Error as e:
        logger.error(f"PostgreSQL error during connection: {e}")
        return f"Database error occurred: {e}"
    except Exception as e:
        logger.error(f"Unexpected error during connection: {e}")
        return f"An unexpected error occurred during connection: {e}"

@tool
def commit() -> str:
    """
    Commit the current database transaction.
    
    Returns:
        str: Success message or error description.
        
    Example:
        >>> exec_query("INSERT INTO users (name) VALUES ('Charlie')")
        >>> commit()
        "Transaction committed successfully"
        
    Note:
        - Required to persist INSERT, UPDATE, DELETE operations
        - Should be called after successful query execution
        - Use rollback operations manually if needed before committing
    """
    global connection
    
    if connection is None:
        return "Error: No database connection established. Use connect() first."
        
    try:
        connection.commit()
        return "Transaction committed successfully"
    except psycopg2.Error as e:
        logger.error(f"PostgreSQL error during commit: {e}")
        return f"Database error occurred during commit: {e}"
    except Exception as e:
        logger.error(f"Unexpected error during commit: {e}")
        return f"An unexpected error occurred during commit: {e}"

@tool
def self_prompt(msg: str) -> str:
    """
    Generate a custom system message to modify AI behavior dynamically.
    
    Args:
        msg (str): The system message content to inject.
        
    Returns:
        str: Response from the AI system or error message.
        
    Example:
        >>> self_prompt("You are now an expert in database optimization")
        "System message processed successfully"
        
    Warning:
        - Use sparingly and with caution
        - Can significantly alter AI behavior and responses
        - Effects may persist for the current conversation context
        - Ensure 'llm_with_tools' and 'state' variables are properly defined
    """
    try:
        # Note: Assumes llm_with_tools and state are defined in the calling context
        result = llm_with_tools.invoke(
            SystemMessage(role="system", content=msg) + state["messages"]
        )
        return "System message processed successfully"
    except NameError as e:
        logger.error(f"Required variables not defined for self_prompt: {e}")
        return f"Configuration error: {e}"
    except Exception as e:
        logger.error(f"Unexpected error in self_prompt: {e}")
        return f"An unexpected error occurred: {e}"